In [4]:
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff 
import pickle
import ts2vg
import scipy
import sys
from diversity_measure import *

In [6]:
raw_data = loadarff("/home/lfrf/ic/time-series-classification/data/LSST/LSST_TRAIN.arff")
df: pd.DataFrame = pd.DataFrame(raw_data[0])
df

,LSST_TRAIN,target
0,"[[17.597, 0.22974, -29.654, -59.21, -72.064, -...",b'6'
1,"[[2.4209, 1.6847, 0.50692, -0.26783, -0.02287,...",b'6'
2,"[[39.688, 163.99, 275.45, 300.36, 222.61, 98.1...",b'6'
3,"[[1.6554, 10.357, 17.141, 15.575, 8.0169, 4.06...",b'6'
4,"[[-15.139, -8.0405, -0.5869, 1.1502, -4.7361, ...",b'6'
...,...,...
2454,"[[16.989, -8.2046, -35.531, -43.974, -25.709, ...",b'95'
2455,"[[8.9566, 4.2787, 1.0259, 5.5224, 15.246, 20.9...",b'95'
2456,"[[503.12, 317.31, 242.74, 257.62, 115.73, 47.8...",b'95'
2457,"[[17.756, 26.275, 31.817, 32.078, 26.981, 18.9...",b'95'


In [7]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_basic_motions

def save_pickle(data: dict, path: str) -> None:
    # open a file, where you ant to store the data
    with open(path, 'wb') as file:
        pickle.dump(data, file)

In [12]:
# Creating graphs and calculating layer differences (only first 5 rows)
aux_df: pd.DataFrame = df.head(5)
visibility_graphs: np.array = np.zeros((5, 6, 36, 36))
for i in np.arange(len(aux_df["LSST_TRAIN"])):
    for j in np.arange(len(aux_df["LSST_TRAIN"].iloc[i])):
        ts = aux_df["LSST_TRAIN"].iloc[i][j]
        aux_ts = np.array([x for x in ts]).astype(np.float_) # gambiarra
        graph: np.array = ts2vg.NaturalVG().build(aux_ts).adjacency_matrix()
        visibility_graphs[i][j] = graph

In [10]:
ranking: np.array = np.zeros((6, 6))
for i in range(visibility_graphs.shape[0]):
    node_distributions: list = []
    transition_matrices: list = []
    for j in range(visibility_graphs[i].shape[0]):
        graph: np.array = visibility_graphs[i][j]
        node_dist: np.array = node_distribution(graph)
        node_distributions.append(node_dist)
        trans_mat: np.array = transition_matrix(graph)
        transition_matrices.append(trans_mat)
    less_contribute: np.array = diversidade(node_distributions, transition_matrices)[1]
less_contribute
        
        

array([1, 2, 5, 4, 0, 3])